In [1]:
import pytest
import deepchem as dc
import tempfile
import numpy as np
import os
print(os.getcwd())
from dataset_mpnn import get_dataset

try:
    import torch
    has_torch = True
except:
    has_torch = False

No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepchem.models.torch_models' (/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/deepchem/models/torch_models/__init__.py)
Skipped loading modules with pytorch-lightning dependency, missing a dependency. No module named 'pytorch_lightning'
Skipped loading some Jax models, missing a dependency. No module named 'jax'


/home/ec2-user/SageMaker/odor analysis/odor-model-pipeline/MPNN


In [10]:
def test_custom_mpnn_model_classification(nb_epoch):
    """
    """
    torch.manual_seed(0)

    # load sample dataset
    # dataset, class_imbalance_ratio = get_dataset(csv_path='assets/GS_LF_sample100.csv')
    dataset, class_imbalance_ratio = get_dataset(csv_path='./../curated_GS_LF_merged_4984.csv')

    # initialize the model
    from custom_mpnn import CustomMPNNModel
    from featurizer import GraphConvConstants

    model = CustomMPNNModel(n_tasks = 138,
                            batch_size=500,
                            learning_rate=0.001,
                            class_imbalance_ratio = class_imbalance_ratio,
                            node_out_feats = 100,
                            edge_hidden_feats = 120,
                            num_step_message_passing = 2,
                            mode = 'classification',
                            number_atom_features = GraphConvConstants.ATOM_FDIM,
                            number_bond_features = GraphConvConstants.BOND_FDIM,
                            n_classes = 1,
                            ffn_hidden_list= [64, 64],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.2,
                            ffn_dropout_at_input_no_act = True,
                            weight_decay = 1e-6,
                            self_loop = False,
                            log_frequency = 1)

    # test
    for epoch in range(nb_epoch):
        loss = model.fit(
          dataset,
          nb_epoch=1,
          max_checkpoints_to_keep=1,
          deterministic=False,
          restore=epoch > 1)
        print(f"epoch {epoch}/{nb_epoch} ; loss = {loss}")
    metric = dc.metrics.Metric(dc.metrics.roc_auc_score)
    scores = model.evaluate(dataset, [metric], n_classes=2)
    print(scores['roc_auc_score'])

In [11]:
test_custom_mpnn_model_classification(10)

number of odor tasks =  138


Failed to featurize datapoint 3399, S. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3409, O. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3414, [Cl-].[K+]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3582, [He]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3583, [Zn]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3602, O.O.O.O.O.O.[Cl-].[Cl-].[Co+2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3615, [Cl-].[NH4+]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3616, [NH4+].[NH4+].[S-2]. Appending empty array
Exception message: tuple index out of range
Failed to featurize datapoint 3716, [Mg+2].[OH-].[OH-]. Appending empty array
Exception

epoch 0/10 ; loss = 0.4611878342674856
epoch 1/10 ; loss = 0.38353842168917146
epoch 2/10 ; loss = 0.3534078645639296
epoch 3/10 ; loss = 0.34004693586138024
epoch 4/10 ; loss = 0.3200361643668389
epoch 5/10 ; loss = 0.3119316474326124
epoch 6/10 ; loss = 0.3021239598371352
epoch 7/10 ; loss = 0.2984927977108005
epoch 8/10 ; loss = 0.2863968907471195
epoch 9/10 ; loss = 0.2810437974944542
0.33139793530034456
